<a href="https://colab.research.google.com/github/ClearKet/Prompt-for-classify-AECO-sector-cost-data/blob/main/COST_DATA_SCHEMA_CLASSIFIER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TEXT CLASSIFIER

In [ ]:
!pip install openai==0.28

In [ ]:
import openai
print(openai.__version__)

In [ ]:
openai.api_key = "INSERT_YOUR_API_KEY"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##TEST DATASET

In this section the dataset composed of cost descriptionsis uploaded

In [ ]:
test_declaratorie_path = 'dataset_file_path'

with open(test_declaratorie_path, 'r') as file:
    dataset = file.read()

print(dataset)

###TAXONOMY DOCUMENTS FOR CLASSIFICATION

Uploading the file containing all the information useful for the realization of the classification of work descriptions, consisting of the fields “genre”, “discipline”, “system”, “object” (thus 4 levels of classification, of which for this study the genre is not considered)


In [ ]:
taxonomy_file_path ='taxonomy_file_path'

with open(taxonomy_file_path, 'r') as f:
    taxonomy = json.load(f)

print(taxonomy)

In [ ]:
Discipline_definition = list({
    (item['DISCIPLINA'], item.get('DISCIPLINA_definizione'))
    for item in taxonomy
})
for disciplina, descrizione in Discipline_definition:
    print(f"[{disciplina}: {descrizione}]")

In [ ]:
codifica_totale = [(item['GENERE'], item['DISCIPLINA'], item['SISTEMA'], item['OGGETTO']) for item in taxonomy]

# Stampa la lista totale delle possibili discipline
for genere, disciplina, sistema, oggetto in codifica_totale:
    print(f"{genere}, {disciplina}, {sistema}, {oggetto}")

In [ ]:
disciplina_output= 'ingegneria strutturale'
system_definition = set(
(item['SISTEMA'], item['SISTEMA_definizione'])
for item in taxonomy
if item.get('DISCIPLINA') == disciplina_output
)

print(system_definition)

{('sistemi complementari', 'Relativi a tutte quelle opere relative a componenti accessori che migliorano il funzionamento e la durabilità, o supportano altre tecniche di consolidamento'), ('sistemi telaio', 'Relativi a tutte quelle opere eseguite per la realizzazione, manutenzione o demolizione delle parti che costituiscono una struttura telaio'), ('sistemi di copertura', 'Relativi a tutte quelle opere eseguite per la realizzazione, manutenzione o demolizione delle parti che costituiscono una copertura'), ('sistema strutturale', 'Relativo a tutte le opere effettuate su strutture di tipo generico'), ('sistemi di ingegneria naturalistica', 'Relativi a tutte quelle opere eseguite per il consolidamento e la difesa del suolo utilizzando metodi che si integrano in modo natuale con la natura'), ('sistemi di consolidamento', 'Relativo a tutte quelle opere di rinforzo o stabilizzazione di strutture esistenti'), ('sistemi di protezione', 'Relativi a tutte quelle opere progettate per il consolida

###EXAMPLES FOR FEWSHOTS

Uploading the file containing the examples for Intormation extraction

In [ ]:
fewshots_example_path = 'example_file_path'

with open(fewshots_example_path, 'r') as f:
    fewshots_examples = json.load(f)

print(fewshots_examples)

##PROMPT PER LA CLASSIFICAZIONE DELLA VOCE

In [ ]:
def generate_completion(prompt, temperature):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "user", "content": prompt}
            ],
             temperature=temperature  # Settaggio della temperatura
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        return f"Errore: {e}"


#OPTIMISED DYNAMIC PROMPT

In [ ]:
def classificator(dataset, Discipline_definition, taxonomy, fewshots_examples, generate_completion, file_path, temperature=0.1):
    """
    Args:
        test_declaratorie (list): list of all cost item descriptions.
        Discipline_definition (str): discipline and their definitions.
        taxonomy (list): containing the list of the systems definition.
        generate_completion (callable): function from OpenAI for performing prompts.
        df_storage (DataFrame): Il DataFrame dove memorizzare i risultati.
        file_path (str): output file path.
     """

#PROMPT FOR DISCIPLINE CLASSIFICATION

    discipline_prompt = (
        f"Data la seguente descrizione di opera edile proveniente dal listino dei prezzi: \n"
        f"'{dataset}', \n\na quale delle seguenti discipline fa riferimento? "
        f"Vorrei che restituissi una risposta secca restituendo solo la disciplina, "
        f"senza la relativa descrizione\n\n{Discipline_definition}."
        )

    print("Prompt generato per disciplina:")
    print(discipline_prompt)

    response_discipline = generate_completion(discipline_prompt, temperature=temperature)
    discipline_output = response_discipline.strip()  # Removes any spaces or newlines

    print("Risposta rilevata:", discipline_output)


#PROMPT FOR SYSTEMS CLASSIFICATION

    # Filtering the systems related to the specific DISICIPLINE
    systems_filtred = set(item['SISTEMA'] for item in taxonomy if item.get('DISCIPLINA') == discipline_output)

    # Cleaning the  il set per stampare senza virgolette
    #sistemi_filtrati_str = "{" + ", ".join(sistemi_filtrati) + "}"
    #print("Sistemi filtrati:", sistemi_filtrati_str)

    # Filtering the systems and their definitions basing on the specific discipline_output
    systems_definitions_filtered = set(
    (item['SISTEMA'], item['SISTEMA_definizione'])
    for item in taxonomy
    if item.get('DISCIPLINA') == discipline_output
    )


    system_prompt = (
        f"Data la seguente descrizione di opera edile proveniente dal listino dei prezzi: \n"
        f"'{dataset}', \n\n"
        f"A quale dei seguenti sistemi, con relativa definizione, appartiene? Vorrei che restituissi una risposta secca, "
        f"restituendo solo il sistema senza la relativa descrizione\n\n{systems_definitions_filtered}."
        )

    print("Prompt generato per sistema:")
    print(system_prompt)

    response_system = generate_completion(system_prompt, temperature=temperature)
    system_output = response_system.strip("'\"") # Removes any spaces or newlines

    print("Risposta rilevata:", system_output)


#PROMPT FOR OBJECT CLASSIFICATION

    # Filtering the objects related to the specific DISICIPLINE and SYSTEM
    objects_filtred = [
    item['OGGETTO'] for item in taxonomy
    if item.get('SISTEMA', '').lower() == system_output.lower() and item.get('DISCIPLINA', '').lower() == discipline_output.lower()
    ]

    objects_filtered_str = ("{" + ", ".join(objects_filtred) + "}")
    print(objects_filtred)

    objects_prompt =(
        f"Data la seguente descrizione di opera edile proveniente dal listino dei prezzi: \n"
        f"{dataset}, la cui disciplina è {discipline_output}, e il sistema è {system_output} \n"
        f"quale delle seguenti categorie di oggetti appartiene? Vorrei che restituissi una risposta secca,"
        f"restituendo solo l'oggetto \n\n{objects_filtered_str}."
        )

    print("Prompt generato per oggetto:")
    print(objects_prompt)

    response_object = generate_completion(objects_prompt, temperature=0.1)
    object_output = response_object
    print("Risposta rilevata:")
    print(object_output)


#PROMPT FOR EXTRACTING ATTRIBUTES

    filtered_data = [item for item in fewshots_examples if item.get("Oggetto") == oggetto_output]

    # Rimuovere le chiavi indesiderate da ogni elemento filtrato
    keys_to_exclude = ["ID", "Oggetto"]  # Specifica le chiavi da escludere
    filtered_data_cleaned = [
    {key: value for key, value in item.items() if key not in keys_to_exclude}
    for item in filtered_data
    ]

    # Mostrare i dati filtrati e puliti
    for item in filtered_data_cleaned:
       print(json.dumps(item, indent=4, ensure_ascii=False))

    prompt_destrutturazione =(
        f"Data la seguente descrizione di opera edile proveniente dal listino dei prezzi: \n'{dataset}',\n"
        f"Identifica le seguenti entità nel testo e struttura le informazioni secondo questi attributi: [Materia: materiale che caratterizza l'oggetto (es. 'ceramica/laterizio'); Geometria/forma: caratteristiche geometriche o di struttura (es. 'piano, nervature parallele'); Dimensione: informazioni dimensionali o di spessore (es. 'spessore tot [cm] = 17'); Incluso e Incluso nel lavoro: elementi o attività comprese nel lavoro; Escluso dal lavoro: attività o materiali esclusi; Specifiche tecniche: eventuali dettagli tecnici aggiuntivi.] \n\n"
        f"Esempio {filtered_data_cleaned}."
        )

    print(prompt_destrutturazione)

    response_destrutturazione = generate_completion(prompt_destrutturazione, temperature=0.1)
    attributi_output = response_destrutturazione  # Salva la risposta in una variabile
    print("Risposta:", attributi_output)


    return discipline_output, sistema_output, oggetto_output, attributi_output


The following part of code is related to the application of the just defined function ont he dataset, storing the results on an excel file and printing them on the output_file_path

In [ ]:
output_file_path = "output_file_path"

In [ ]:
current_id = 1

for current_id, description in enumerate(dataset, start=1):
    # Chiama la funzione destrutturazione
    disciplina_output, sistema_output, oggetto_output, attributi_output  = classificator(description, Discipline_definition, taxonomy, generate_completion, output_file_path, temperature=0.1)


    # Crea una nuova riga per il DataFrame
    new_row = {
        "ID": current_id,  # Assegna un ID progressivo
        "CODE": f"CODE_{current_id}",  # Provide a unique ID
        "DISCIPLINA": disciplina_output,
        "SISTEMA": sistema_output,
        "OGGETTO": oggetto_output,
        "ATTRIBUTI": attributi_output,
    }

    # Increases the ID for the next iteration
    current_id += 1

    # Add the new row to the DataFrame
    new_row_df = pd.DataFrame([new_row])
    df_storage = pd.concat([df_storage, new_row_df], ignore_index=True)

    # Save the updated DataFrame to file
    df_storage.to_excel(output_file_path, index=False)
    print(f"Risultati salvati per descrizione: '{description}'")

Prompt generato per disciplina:
Data la seguente descrizione di opera edile proveniente dal listino dei prezzi: 
'Isolamento termico con microventilazione o con ventilazione di coperture, realizzato a tetto caldo, con applicazione all'estradosso delle falde di pannelli di schiuma poliuretanica prodotta con gas senza CFC e HCFC, con rivestimento superficiale permeabile al vapore e impermeabile all'acqua, dotati di listelli di legno incorporati nella schiuma. Conforme alla Norma UNI EN 13165, conduttività termica W/mK 0,029. Spessore 50 mm. Compresi: tagli e relative sigillature, adattamenti, fissaggi di qualsiasi tipo a qualsiasi struttura, raccordi, assistenza muraria, piani di lavoro.', 

a quale delle seguenti discipline fa riferimento? Vorrei che restituissi una risposta secca restituendo solo la disciplina, senza la relativa descrizione

[('ergotecnica', 'Disciplina relativa a tutte quelle opere svolte in cantiere a supporto della realizzazione delle lavorazioni'), ('suolo', 'Relat